In [6]:
from os import listdir
from pickle import dump
import tensorflow as tf
import pandas as pd
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
import cv2
import numpy as np

In [2]:
 
tf.__version__

'2.1.0'

In [8]:
model = VGG16()

553467904/553467096 [==============================] - 122s 0us/step


# Loading the Data

In [7]:

image_dataframe=pd.read_csv('/home/pytorch_ashish/deep_learning/data/alt_text_data_all/image_location_venky.csv')
cengage= pd.read_csv('/home/pytorch_ashish/deep_learning/data/alt_text_data_all/cengage_production_data.csv')
pearson= pd.read_csv('/home/pytorch_ashish/deep_learning/data/alt_text_data_all/pearson_production_data.csv')
wiley=pd.read_csv('/home/pytorch_ashish/deep_learning/data/alt_text_data_all/wiley_production_data.csv')
wiley.rename(columns={'image_names':'Name'}, inplace=True)
pearson.rename(columns={'image_names':'Name'}, inplace=True)
cengage.rename(columns={'image_names':'Name'}, inplace=True)

image_dataframe=pd.merge(image_dataframe, cengage, on='Name', how='left')
image_dataframe.drop(['image_path', 'alt_text_html', 'Unnamed: 0'], axis=1, inplace=True)

image_dataframe=pd.merge(image_dataframe, pearson, on='Name', how='left')
image_dataframe["alt_text"] = image_dataframe["alt_text_x"].astype(str) + " "+image_dataframe["alt_text_y"].astype(str)
image_dataframe['subject_area'] = image_dataframe['subject_area_x'].astype(str) + " "+ image_dataframe['subject_area_y'].astype(str)
image_dataframe.drop(['alt_text_x', "alt_text_y", 'subject_area_x', 'subject_area_y', 'alt_text_html', 'image_path'], axis=1, inplace=True)

image_dataframe=pd.merge(image_dataframe, wiley, on='Name', how='left')
image_dataframe["alt_text"] = image_dataframe["alt_text_x"].astype(str) + " "+ image_dataframe["alt_text_y"].astype(str)
image_dataframe['subject_area'] = image_dataframe['subject_area_x'].astype(str) + " "+ image_dataframe['subject_area_y'].astype(str)
image_dataframe.drop(['alt_text_x', "alt_text_y", 'subject_area_x', 'subject_area_y', 'alt_text_html', 'image_path'], axis=1, inplace=True)

banned = ['nan']
f = lambda x: ' '.join([item for item in x.split() if item not in banned])
image_dataframe["subject_area"] = image_dataframe["subject_area"].apply(f)
image_dataframe["alt_text"] = image_dataframe["alt_text"].apply(f)
image_dataframe = image_dataframe[image_dataframe.subject_area != '']
image_dataframe.subject_area=image_dataframe.subject_area.replace({"other other": "others", "other": "others"})
image_dataframe.reset_index(drop=True, inplace=True)

include = ['maths', 'science', 'others', 'emss', 'canada', 'engineering', 'business', 'history']
indexNames=[]
for i in range(image_dataframe.shape[0]):
    if(image_dataframe.loc[i]['subject_area'] not in include):
        indexNames.append(i)
image_dataframe.drop(labels=indexNames, inplace=True)
image_dataframe.reset_index(drop=True, inplace=True)

image_dataframe.head()

,Name,Path,Height,Width,alt_text,subject_area
0,9781337111348_11348_ch02_2.1_017-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,131,181,The image consists of a Minitab output. Visual...,maths
1,9781285194790_9781285194790-551-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,550,593,A map shows the Second Iraq War. Majority popu...,history
2,9781133947257_9781133947257_ch01_65-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,101,184,In the Sherlock Holmes mystery The Final Solut...,maths
3,9781305652231_52231_ch01_f022-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,24,188,The image consists of a number line. The numbe...,maths
4,ceng_image205.png,/home/pytorch_ashish/deep_learning/data/alt_te...,131,268,The image consists of a visual representation ...,maths


In [8]:
image_dataframe.shape

(173778, 6)

In [9]:
image_dataframe['subject_area'].value_counts()

maths          146839
science         10820
others           5116
emss             3602
canada           2852
engineering      1854
business         1681
history          1014
Name: subject_area, dtype: int64

In [10]:
image_dataframe.subject_area.unique()

array(['maths', 'history', 'science', 'engineering', 'business', 'others',
       'emss', 'canada'], dtype=object)

In [11]:
image_dataframe.isna().sum()

Name            0
Path            0
Height          0
Width           0
alt_text        0
subject_area    0
dtype: int64

In [13]:
X= image_dataframe.loc[:,:]
y= image_dataframe.loc[:,'subject_area']


In [14]:
X.head()

,Name,Path,Height,Width,alt_text,subject_area
0,9781337111348_11348_ch02_2.1_017-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,131,181,The image consists of a Minitab output. Visual...,maths
1,9781285194790_9781285194790-551-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,550,593,A map shows the Second Iraq War. Majority popu...,history
2,9781133947257_9781133947257_ch01_65-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,101,184,In the Sherlock Holmes mystery The Final Solut...,maths
3,9781305652231_52231_ch01_f022-t2.png,/home/pytorch_ashish/deep_learning/data/alt_te...,24,188,The image consists of a number line. The numbe...,maths
4,ceng_image205.png,/home/pytorch_ashish/deep_learning/data/alt_te...,131,268,The image consists of a visual representation ...,maths


In [15]:
y.head()

0      maths
1    history
2      maths
3      maths
4      maths
Name: subject_area, dtype: object

In [22]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [23]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(130333, 6) (130333,) (43445, 6) (43445,)


In [24]:
def gray(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = np.expand_dims(gray_image, axis=-1)
    return gray_image
INPUT_DIM = (128,128,3)
BATCH_SIZE = 32

In [19]:
trdata = ImageDataGenerator(rescale=1./255,  preprocessing_function=gray)
traindata = trdata.flow_from_dataframe(dataframe = X_train
                                         , directory = None
                                         , x_col='Path'
                                         , y_col='subject_area'
                                         , target_size = (224,224)
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         #, classes = y_train_en
                                         , class_mode = 'categorical'
                                            )


Found 130333 validated image filenames belonging to 8 classes.


In [20]:
tsdata = ImageDataGenerator(rescale=1./255,  preprocessing_function=gray)
testdata = tsdata.flow_from_dataframe(dataframe = X_test
                                         , directory = None
                                         , x_col='Path'
                                         , y_col='subject_area'
                                         , target_size = (224,224)
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         #, classes = y_train_en
                                         , class_mode = 'categorical'
                                            )

Found 43445 validated image filenames belonging to 8 classes.


In [28]:
vggmodel.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [23]:
vggmodel = VGG16(weights='imagenet', include_top=True)
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [24]:
for layers in (vggmodel.layers)[:19]:
    print(layers)
    layers.trainable = False

In [33]:
from tensorflow.keras.layers import Dense
X= vggmodel.layers[-2].output
predictions = Dense(8, activation="softmax")(X)
model_final = Model(vggmodel.input, predictions)

In [34]:
model_final.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [60]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard, TerminateOnNaN, EarlyStopping
import os
from datetime import datetime
RUN_FOLDER='/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/' 
logdir = os.path.join(RUN_FOLDER, "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=logdir)

#checkpoint_filepath=os.path.join(run_folder, "model.h5")
checkpoint1 = ModelCheckpoint(os.path.join(RUN_FOLDER, "model.h5"), monitor='val_loss',  save_best_only=True, verbose=1, mode= 'min')
checkpoint2 = ModelCheckpoint(os.path.join(RUN_FOLDER, 'weights.h5'), save_weights_only = True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, mode='min', verbose=1)
EarlyStop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=31, verbose=1, mode='min', restore_best_weights=True)
terminate = TerminateOnNaN()

callbacks_list = [checkpoint1, checkpoint2, reduce_lr, tensorboard_callback, EarlyStop, terminate]

In [22]:
NUM_IMAGES= X_train.shape[0]
BATCH_SIZE=32
EPOCHS=100
VAL_NUM_IMAGES = X_test.shape[0]

In [23]:
print(VAL_NUM_IMAGES, NUM_IMAGES)

43445 130333


In [63]:
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.0001), metrics=["accuracy"])

In [67]:
model_final.fit(
            x = traindata
            , shuffle = True
            , epochs = EPOCHS
            , initial_epoch = 0
            , callbacks = callbacks_list
            , steps_per_epoch=NUM_IMAGES / BATCH_SIZE
            , workers= 7
            , use_multiprocessing=False
            , validation_data= testdata
            , validation_steps= VAL_NUM_IMAGES / BATCH_SIZE
            )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 4072.90625 steps, validate for 1357.65625 steps
Epoch 1/100
4072/4072 [============================>.] - ETA: 0s - loss: 0.3427 - accuracy: 0.8941
Epoch 00001: val_loss improved from inf to 0.33799, saving model to /home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/model.h5

Epoch 00001: saving model to /home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/weights.h5
4073/4072 [==============================] - 1639s 402ms/step - loss: 0.3427 - accuracy: 0.8941 - val_loss: 0.3380 - val_accuracy: 0.8933
Epoch 2/100
4072/4072 [============================>.] - ETA: 0s - loss: 0.2411 - accuracy: 0.9226
Epoch 00002: val_loss improved from 0.33799 to 0.27053, saving model to /home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/model.h5

Epoch 00002: saving model to /home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/weights.h5
4073/4072 [=====

KeyboardInterrupt: 

In [68]:
filename=RUN_FOLDER+'val_acc_9280_model.h5'
model_final.save(filename)

# Retraining the Model

In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow.keras import layers
#R_LOSS_FACTOR = 10000
#import numpy as np

In [2]:
RUN_FOLDER='/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/'

In [3]:
new_model = load_model("/home/pytorch_ashish/deep_learning/data/alt_text_data_all/models/NIC_model/val_acc_9280_model.h5")

In [4]:
learning_rate=K.eval(new_model.optimizer.lr)
learning_rate

1.6e-07

In [16]:
new_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=learning_rate), metrics=["accuracy"])

In [17]:
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [20]:
for layers in (new_model.layers)[:]:
    try:
        print(layers.get_config()['trainable'])  #
    except:
        pass
    #layers.trainable = True

False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True


In [19]:
for layers in (new_model.layers)[11:]:
    try:
        if(layers.get_config()['trainable']==False):
            layers.trainable = True#
    except:
        pass
    

In [44]:
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

# Checking the ouput dtype and shape for the image data generator to include it in tf.data pipeline

In [27]:
tr_images, tr_labels= next(trdata.flow_from_dataframe(dataframe = X_train
                                         , directory = None
                                         , x_col='Path'
                                         , y_col='subject_area'
                                         , target_size = (224,224)
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         #, classes = y_train_en
                                         , class_mode = 'categorical'
                                            ))

Found 130333 validated image filenames belonging to 8 classes.


In [28]:
print(tr_images.dtype, tr_images.shape)
print(tr_labels.dtype, tr_labels.shape)

float32 (32, 224, 224, 3)
float32 (32, 8)


In [45]:
images, labels= next(tsdata.flow_from_dataframe(dataframe = X_test
                                         , directory = None
                                         , x_col='Path'
                                         , y_col='subject_area'
                                         , target_size = (224,224)
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         #, classes = y_train_en
                                         , class_mode = 'categorical'
                                            ))

Found 43445 validated image filenames belonging to 8 classes.


In [46]:
print(images.dtype, images.shape)
print(labels.dtype, labels.shape)

float32 (32, 224, 224, 3)
float32 (32, 8)


# Creating tf.data and training the model

In [32]:
trdata = ImageDataGenerator(rescale=1./255,  preprocessing_function=gray)
traindata=trdata.flow_from_dataframe(dataframe = X_train
                                         , directory = None
                                         , x_col='Path'
                                         , y_col='subject_area'
                                         , target_size = (224,224)
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         #, classes = y_train_en
                                         , class_mode = 'categorical'
                                            )

Found 130333 validated image filenames belonging to 8 classes.


In [52]:
tsdata = ImageDataGenerator(rescale=1./255,  preprocessing_function=gray)
testdata=tsdata.flow_from_dataframe(dataframe = X_test
                                         , directory = None
                                         , x_col='Path'
                                         , y_col='subject_area'
                                         , target_size = (224,224)
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         #, classes = y_train_en
                                         , class_mode = 'categorical'
                                            )

Found 43445 validated image filenames belonging to 8 classes.


In [43]:
print(X_train.shape)
print(int(round(X_train.shape[0]/BATCH_SIZE ,0)))
print(BATCH_SIZE)

(130333, 6)
4073
32


In [53]:
print(X_test.shape)
print(int(round(X_test.shape[0]/BATCH_SIZE ,0)))
print(BATCH_SIZE)

(43445, 6)
1358
32


In [56]:
def callable_gen():
    #for i in range(int(round(X_train.shape[0]/BATCH_SIZE ,0))):
    img, label =next(traindata)
    k = (img, label)
    yield k

In [57]:
def callable_gen_test():
    #for i in range(int(round(X_test.shape[0]/BATCH_SIZE ,0))):
    img, label =next(testdata)
    kt = (img, label)
    yield kt

In [58]:
#dataframe = X_test
#directory = None
#x_col='Path'
#y_col='subject_area'
#target_size = (224,224)
#batch_size = BATCH_SIZE
#shuffle = True
#, classes = y_train_en
#class_mode = 'categorical'
#trdata = ImageDataGenerator(rescale=1./255,  preprocessing_function=gray)
ds = tf.data.Dataset.from_generator(
    callable_gen,
    output_types=(tf.float32, tf.float32), 
    output_shapes=([32,224,224,3], [32,8])
)
ds

<FlatMapDataset shapes: ((32, 224, 224, 3), (32, 8)), types: (tf.float32, tf.float32)>

In [59]:
ds_test = tf.data.Dataset.from_generator(
    callable_gen_test,
    output_types=(tf.float32, tf.float32), 
    output_shapes=([32,224,224,3], [32,8])
)
ds_test

<FlatMapDataset shapes: ((32, 224, 224, 3), (32, 8)), types: (tf.float32, tf.float32)>

In [69]:
ds

<FlatMapDataset shapes: ((32, 224, 224, 3), (32, 8)), types: (tf.float32, tf.float32)>

In [65]:
mui=int(round(X_train.shape[0]/BATCH_SIZE ,0))
muv=int(round(X_test.shape[0]/BATCH_SIZE ,0))

In [66]:
new_model.fit(
            x = callable_gen
            , shuffle = True
            , epochs = 100
            , initial_epoch = 0
            , callbacks = callbacks_list
            , steps_per_epoch=mui
            , workers= 7
            , use_multiprocessing=True
            , validation_data= callable_gen_test
            , validation_steps= muv
            )

ValueError: Failed to find data adapter that can handle input: <class 'function'>, <class 'NoneType'>